In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
from tensorflow.keras.models import load_model
import pickle

### Data Ingestion

In [3]:
data_path : str=os.path.join("data","bixi_wrt_cal_15min.csv")
bixi_dat = pd.read_csv(data_path, parse_dates = ['index'],  index_col=0)
bixi_dat.rename(columns={'index':'timestamp'}, inplace=True)
bixi_dat

,timestamp,com1,com2,com3,com4,com5,com6,business_day,tod_afternoon,tod_afternoon_rush,tod_evening,tod_lunch_time,tod_morning,tod_morning_rush,tod_night,Temperature,Precipitation,Wind Speed,Relative Humidity
0,2017-04-15 00:00:00,8.0,5.0,4.0,11.0,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.3,0.0,9.0,57.0
1,2017-04-15 00:15:00,1.0,5.0,3.0,8.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.3,0.0,9.0,57.0
2,2017-04-15 00:30:00,2.0,5.0,6.0,9.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.3,0.0,9.0,57.0
3,2017-04-15 00:45:00,3.0,3.0,4.0,2.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.3,0.0,9.0,57.0
4,2017-04-15 01:00:00,7.0,4.0,3.0,6.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.3,0.0,7.0,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81137,2021-05-31 22:00:00,29.0,18.0,33.0,30.0,77.0,22.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,13.5,0.0,13.0,74.0
81138,2021-05-31 22:15:00,11.0,9.0,15.0,15.0,40.0,9.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,13.5,0.0,13.0,74.0
81139,2021-05-31 22:30:00,21.0,32.0,33.0,24.0,60.0,6.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,13.5,0.0,13.0,74.0
81140,2021-05-31 22:45:00,21.0,29.0,27.0,39.0,55.0,9.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,13.5,0.0,13.0,74.0


In [9]:
# Bike-sharing demand for each community from 1 to 6
demand_features = ['com1', 'com2', 'com3', 'com4', 'com5', 'com6']

# Bike sharing demand for each community over time
bixi_demand = bixi_dat[demand_features].values
print(bixi_demand.shape)

(81309, 6)


In [11]:
from tensorflow.keras.models import load_model
import pickle

model_path: str=os.path.join("artifacts",'TreNet_LSTM_model.h5')
model_artifacts_path: str=os.path.join("artifacts",'artifacts.pkl')
# load model
model = load_model(model_path)
# summarize model
model.summary()
# load the scaler
artifacts = pickle.load(open(model_artifacts_path, 'rb'))
scaler = artifacts['scaler']
history_length = artifacts['history_length']
n_seq = artifacts['n_seq']
n_steps = artifacts['n_steps']
n_features = artifacts['n_features']

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, None, 8, 128)     896       
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, None, 8, 128)     16512     
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, None, 1024)       0         
 tributed)                                                       
                                                                 
 lstm (LSTM)                 (None, 50)                215000    
                                                                 
 dense (Dense)               (None, 64)                3264      
                                                        

2023-07-11 19:07:54.270361: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-11 19:07:54.272322: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-11 19:07:54.273883: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [12]:
def sequence_new_input (sequence, history_length):
    X = list()
    for i in range(len(sequence)):
    # find the end of this pattern
        end_ix = i + history_length
    # check if we are beyond the sequence
        if end_ix > len(sequence):
            break
    # gather input and output parts of the pattern
        seq_x = sequence[i:end_ix]
        X.append(seq_x)
    return np.array(X)

# split a series dataset into supervised learning
def model_input_predict(dat, scaler=scaler, history_length=history_length , n_seq=n_seq, n_steps=n_steps, n_features=n_features):
    scaled_dat = scaler.transform(dat)
    m, n = scaled_dat.shape
    # e.g., if history_length = 7, BIXI demand of past 7*15 minutes
    scaled_X = np.zeros((m-history_length+1,history_length, n))
    for i in range(0,n):
        X = sequence_new_input(scaled_dat[:,i],history_length)
        scaled_X[:,:,i] = X
    return scaled_X.reshape((X.shape[0], n_seq, n_steps, n_features))


# Goal of the model:
#  Predict bike sharing demand at a specified time in the future.
#  Eg. We want to predict the number of bixi pickups for each community in 15 minutes from now.
#  We can use all the values from t-1, t-2, t-3, .... t-history_length to predict t+10

def create_ts(dataset,
              step_size,
              ref_index,
              history_length = history_length):
  ''' 
  params step_size (int): #of timestamps to predict after the ref index
  params ref_index(int): The index of timestamp when, prediction starts. Eg. If ref_index = 100, then predict the next timestep after the ref
  params history_length (int): The history length or lag
  '''
  assert step_size > 0
  if ref_index is None:
      ref_index = len(dataset) -1
  assert ref_index > history_length, 'WARNING: ref index can not be lower than history lenght! '
  assert ref_index-history_length-step_size + 1 >= 0, 'WARNING: Pass different combination of values! '

  indices = range(ref_index - history_length - step_size + 2, ref_index + 1)
  dat = dataset[indices]
  ts_X = model_input_predict(dat)
  return ts_X

In [14]:
X_test =  create_ts(bixi_demand, step_size = 10, ref_index=1000)
scaler.inverse_transform(model.predict(X_test))

1/1 [==============================] - 0s 24ms/step


array([[ 49.99423 ,  60.879364,  77.73513 ,  54.333015, 139.4887  ,
         19.786749],
       [ 55.986538,  69.77666 ,  83.93539 ,  67.6695  , 165.25774 ,
         23.026686],
       [ 50.38206 ,  64.94151 ,  70.707016,  66.69024 , 151.38625 ,
         20.519547],
       [ 42.599766,  54.4825  ,  53.377   ,  61.55655 , 134.48888 ,
         16.47459 ],
       [ 35.96427 ,  45.69069 ,  45.462914,  52.185417, 116.82039 ,
         13.928745],
       [ 25.68164 ,  31.396933,  35.431534,  36.523712,  86.6236  ,
         10.70862 ],
       [ 23.57336 ,  28.49718 ,  29.783274,  32.604908,  71.922585,
          9.408033],
       [ 22.148764,  26.420107,  27.20408 ,  30.337852,  66.218025,
          8.582592],
       [ 16.782804,  17.835035,  19.9993  ,  23.814318,  49.896404,
          6.941366],
       [ 18.472336,  15.501308,  18.879639,  22.65854 ,  47.705975,
          6.59379 ]], dtype=float32)